# Title

## Setup

In [7]:
# --- Configture Notebook ------
# show all outputs of cell
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=100,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# enable automatic reloading
%load_ext autoreload
%autoreload 2

import plotly.express as px
from sklearn.pipeline import FunctionTransformer, Pipeline
from arxiv_article_classifier.data.load import load_processed_data
import pandas as pd


from pathlib import Path

from pandas.core.base import PandasObject
from arxiv_article_classifier.utils import display_fully

PandasObject.display_fully = display_fully

DATAFOLDER = Path().cwd().parent / "data"



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Constants
DATAFOLDER_INTERIM = Path().cwd().parent / "data" / "interim"
DATAFOLDER_PROCESSED = Path().cwd().parent / "data" / "processed"

In [9]:
(X_train_raw, _, _, y_train, _, _), labels = load_processed_data(DATAFOLDER_INTERIM)

In [10]:
X_train_raw

array(['Vision-based formation control systems recently have attracted attentions\nfrom both the research community and the industry for its applicability in\nGPS-denied environments. The safety assurance for such systems is challenging\ndue to the lack of formal specifications for computer vision systems and the\ncomplex impact of imprecise estimations on distributed control. We propose a\ntechnique for safety assurance of vision-based formation control. Our technique\ncombines (1) the construction of a piecewise approximation of the worst-case\nerror of perception and (2) a classical Lyapunov-based safety analysis of the\nconsensus control algorithm. The analysis provides the ultimate bound on the\nrelative distance between drones. This ultimate bound can then be used to\nguarantee safe separation of all drones. We implement an instance of the\nvision-based control system on top of the photo-realistic AirSim simulator. We\nconstruct the piecewise approximation for varying perception 

In [11]:
import numpy as np
from arxiv_article_classifier.data.make_processed_data_bow import (
    LATEX_REGEX,
    LINEBREAK_REGEX,
    NLP,
    PUNCTUATION_DELETION_TABLE,
    STOPLIST,
    delete_regular_expression,
    lemmatize_document,
    remove_stopwords,
)


linebreak_cleaner = FunctionTransformer(
    lambda X: np.array([delete_regular_expression(x, LINEBREAK_REGEX) for x in X])
)

lower_case_converter = FunctionTransformer(lambda X: np.array([x.lower() for x in X]))
whitespace_deleter = FunctionTransformer(lambda X: np.array([" ".join(x.split()) for x in X]))

lemmatizer = FunctionTransformer(lambda X: np.array([lemmatize_document(x, NLP) for x in X]))

punctuation_deleter = FunctionTransformer(
    lambda X: np.array([x.translate(PUNCTUATION_DELETION_TABLE) for x in X])
)

stopword_remover = FunctionTransformer(
    lambda X: np.array([remove_stopwords(x, STOPLIST) for x in X])
)

latex_remover = FunctionTransformer(
    lambda X: np.array([delete_regular_expression(x, LATEX_REGEX) for x in X])
)

cleaning_pipeline = Pipeline(
    [
        ("clean_linebreaks", linebreak_cleaner),
        ("remove_latex", latex_remover),
        ("lemmatize", lemmatizer),
        ("convert_to_lowercase", lower_case_converter),
        ("delete_punctuation", punctuation_deleter),
        ("delete_whitespace", whitespace_deleter),
        ("remove_stopwords", stopword_remover),
    ]
)

In [12]:
X_train = cleaning_pipeline.transform(X_train_raw)

In [13]:
abstracts = (
    pd.DataFrame(y_train, columns=labels)
    .assign(abstract=X_train)
    .assign(nwords=lambda df: df["abstract"].map(lambda abstract: len(abstract.split())))
)

px.bar(abstracts["nwords"].value_counts())

Abstracts have between 7 and 213 words after stop-word removal and the distribution does not show any obvious outliers. Let's have a quick look at both ends if they look reasonable.

In [17]:
abstracts[abstracts["nwords"] < 20][["abstract"]].display_fully()

,abstract
406,paper show initial stage development first principle formal logic characterise explore issue broadly define idea veracity
745,work consider estimation method sparse poisson model inspire 1 provide novel sign consistency result mild condition
1027,short note establish positionality mean payoff game infinite game graph construct well found monotone universal graph
1214,note uncover three connection metric distortion problem voting method axiom social choice literature
1628,present optimal transport framework perform regression covariate response probability distribution compact euclidean subset
3318,paper argue synthetic datum produce differentially private generative model sufficiently anonymize therefore anonymous datum regulatory compliant
3932,paper prove dalembert lagrange principle point masse use lagrange mach mechanical construction yield weighted balancing condition unit vector
4003,prove coefficient sum square entry symmetric matrix
4176,standard fractional projection extend binary two mode network weight two mode network interesting property extended projection prove
4700,work tackle problem online camera robot pose estimation single view successive frame image sequence crucial task robot interact world


In [15]:
abstracts[abstracts["nwords"] > 200][["abstract"]].display_fully()

,abstract
1341,complexity increasingly tight coupling supply chain pose major logistical challenge lead company another challenge lead company -- pressure consumer critical public legislative measure supply chain law -- take responsibility supplier labour standard paper discuss new approach lead company use try address challenge algorithmic prediction business risk also environmental social risk describe technical cultural condition algorithmic prediction explain -- perspective lead company -- help address challenge develop scenario kind social consequence algorithmic prediction use lead company scenario derive policy option different stakeholder group help develop algorithmic prediction towards improve labour standard worker voice -- die komplexitat und zunehmend enge kopplung vieler lieferketten stellt eine grosse logistische herausforderung fur leitunternehmen dar eine weitere herausforderung besteht darin dass leitunternehmen -- gedrangt durch konsument innen eine kritische offentlichkeit und gesetzgeberische massnahman wie die lieferkettengesetze -- starker al bisher verantwortung fur arbeitsstandards ihren zulieferbetrieben ubernehmen mussen diesem beitrag diskutieren wir einen neuen ansatz mit dem leitunternehmen versuchen diese herausforderungen zu bearbeiten die algorithmische vorhersage von betriebswirtschaftlichen aber auch okologischen und sozialen risiken wir beschreiben die technischen und kulturellen bedingungen fur algorithmische vorhersage und erklaren wie diese -- aus perspektive von leitunternehmen -- bei der bearbeitung beider herausforderungen hilft anschliessend entwickeln wir szenarien wie und mit welchen sozialen konsequenzen algorithmische vorhersage durch leitunternehmen eingesetzt werden kann aus den szenarien leiten wir handlungsoptionen fur verschiedene stakeholder gruppen ab die dabei helfen sollen algorithmische vorhersage sinne einer verbesserung von arbeitsstandards und workers voice weiterzuentwickeln
1483,paper integrate nonlinear manifold reduced order model nm roms domain decomposition dd nm roms approximate fom state nonlinear manifold train shallow sparse autoencoder use fom snapshot datum nm rom advantageous linear subspace roms ls roms problem slowly decay kolmogorov -width however number nm rom parameter need train scale size fom moreover extreme scale problem storage high dimensional fom snapshot alone make rom training expensive alleviate training cost paper apply dd fom compute nm rom subdomain couple obtain global nm rom approach several advantage subdomain nm rom train parallel involve parameter train global nm rom require small subdomain fom dimensional training datum training subdomain nm rom tailor subdomain specific feature fom shallow sparse architecture autoencoder use subdomain nm rom allow application hyper reduction hr reduce complexity cause nonlinearity yield computational speedup nm rom paper provide first application nm rom hr dd problem particular detail algebraic dd formulation fom train nm rom hr subdomain develop sequential quadratic programming sqp solver evaluate coupled global nm rom theoretical convergence result sqp method priori posteriori error estimate dd nm rom hr provide propose dd nm rom hr approach numerically compare dd ls rom hr 2d steady state burgers equation show order magnitude improvement accuracy propose dd nm rom dd ls rom
1837,promising solution improve communication quality unmanned aerial vehicle uav widely integrate wireless network paper sake enhance message exchange rate user1 u1 user2 u2 intelligent reflective surface irs-and uav- assist two way amplify forward af relay wireless system propose u1 u2 communicate via uav mount irs af relay besides optimization problem maximize minimum rate cast variable namely af relay beamforme matrix irs phase shift two time slot need optimize achieve maximum rate low complexity alternately iterative ai scheme base zero forcing successive convex approximation lc zf sca algorithm put forward expression a

This looks all reasonable apart from that one, which gives its abstract both in English and in German. Let's ignore this for now though and proceed to create the dataset and train our first baseline algorithm. 

In [18]:
X_train_raw[1341]

'The complexity and increasingly tight coupling of supply chains poses a major\nlogistical challenge for leading companies. Another challenge is that leading\ncompanies -- under pressure from consumers, a critical public and legislative\nmeasures such as supply chain laws -- have to take more responsibility than\nbefore for their suppliers\' labour standards. In this paper, we discuss a new\napproach that leading companies are using to try to address these challenges:\nalgorithmic prediction of business risks, but also environmental and social\nrisks. We describe the technical and cultural conditions for algorithmic\nprediction and explain how -- from the perspective of leading companies -- it\nhelps to address both challenges. We then develop scenarios on how and with\nwhat kind of social consequences algorithmic prediction can be used by leading\ncompanies. From the scenarios, we derive policy options for different\nstakeholder groups to help develop algorithmic prediction towards im

This looks reasonable. Let's proceed to create the dataset and train our first baseline algorithm. 

In [16]:
# uncomment the following code or use make processed-data instead

# from arxiv_article_classifier.data.make_processed_data_bow import convert_interim_to_processed_data
#
# datafolder_processed_bow = DATAFOLDER_PROCESSED / "bow-model"
# datafolder_processed_bow.mkdir(exist_ok=True)
# convert_interim_to_processed_data(DATAFOLDER_INTERIM, datafolder_processed_bow)